# Web Scraping - Lab

## Introduction

Now that you've seen a more extensive example of developing a web scraping script, it's time to further practice and formalize that knowledge by writing functions to parse specific pieces of information from the web page and then synthesizing these into a larger loop that will iterate over successive web pages in order to build a complete dataset.

## Objectives

You will be able to:

* Navigate HTML documents using Beautiful Soup's children and sibling relations
* Select specific elements from HTML using Beautiful Soup
* Use regular expressions to extract items with a certain pattern within Beautiful Soup
* Determine the pagination scheme of a website and scrape multiple pages

## Lab Overview

This lab will build upon the previous lesson. In the end, you'll look to write a script that will iterate over all of the pages for the demo site and extract the title, price, star rating and availability of each book listed. Building up to that, you'll formalize the concepts from the lesson by writing functions that will extract a list of each of these features for each web page. You'll then combine these functions into the full script which will look something like this:  

```python
df = pd.DataFrame()
for i in range(2,51):
    url = "http://books.toscrape.com/catalogue/page-{}.html".format(i)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    new_titles = retrieve_titles(soup)
    new_star_ratings = retrieve_ratings(soup)
    new_prices = retrieve_prices(soup)
    new_avails = retrieve_avails(soup)
    ...
 ```

## Retrieving Titles

To start, write a function that extracts the titles of the books on a given page. The input for the function should be the `soup` for the HTML of the page.

In [5]:
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup

In [18]:
html_page = requests.get("http://books.toscrape.com/catalogue/page-1.html")
soupx = BeautifulSoup(html_page.content, 'html.parser')

In [35]:
print(soupx.prettify())

<!DOCTYPE html>
<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!-->
<html class="no-js" lang="en-us">
 <!--<![endif]-->
 <head>
  <title>
   All products | Books to Scrape - Sandbox
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>
  <meta content="24th Jun 2016 09:30" name="created"/>
  <meta content="" name="description"/>
  <meta content="width=device-width" name="viewport"/>
  <meta content="NOARCHIVE,NOCACHE" name="robots"/>
  <!-- Le HTML5 shim, for IE6-8 support of HTML elements -->
  <!--[if lt IE 9]>
        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>
        <![endif]-->
  <link href="../static/oscar/favicon.ico" rel="shortcut icon"/>
  <link href="../static/oscar/css/styles.css" rel="stylesheet" typ

In [20]:
def retrieve_titles(soup):
    all_books = soup.find('div', class_="alert alert-warning").nextSibling.nextSibling
    return [h3.find('a').attrs['title'] for h3 in all_books.findAll('h3')]

## Retrieve Ratings

Next, write a similar function to retrieve the star ratings on a given page. Again, the function should take in the `soup` from the given HTML page and return a list of the star ratings for the books. These star ratings should be formatted as integers.

In [34]:
def retrieve_ratings(soup):
    regex = re.compile("star-rating (.*)")
    stars = soup.findAll('p', {"class": regex})
    numbers = {"One":1, "Two":2, "Three":3, "Four":4, "Five":5}
    return [numbers[x.attrs['class'][1]] for x in stars]

retrieve_ratings(soupx)

[3, 1, 1, 4, 5, 1, 4, 3, 4, 1, 2, 4, 5, 5, 5, 3, 1, 1, 2, 2]

## Retrieve Prices

Now write a function to retrieve the prices on a given page. The function should take in the `soup` from the given page and return a list of prices formatted as floats.

In [38]:
def retrieve_prices(soup):
    prices = soup.findAll('p', {"class": 'price_color'})
    return [float(x.text[1:]) for x in prices]
retrieve_prices(soupx)

[51.77,
 53.74,
 50.1,
 47.82,
 54.23,
 22.65,
 33.34,
 17.93,
 22.6,
 52.15,
 13.99,
 20.66,
 17.46,
 52.29,
 35.02,
 57.25,
 23.88,
 37.59,
 51.33,
 45.17]

## Retrieve Availability

Write a function to retrieve whether each book is available or not. The function should take in the `soup` from a given html page and return a list of the availability for each book.

In [43]:
def retrieve_availabilities(soup):
    avails = soup.findAll('p', {"class": "instock availability"})
    boolize = {'In stock': True}
    return [x.text.strip() == 'In stock' for x in avails]
retrieve_availabilities(soupx)

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

## Create a Script to Retrieve All the Books From All 50 Pages

Finally, write a script to retrieve all of the information from all 50 pages of the books.toscrape.com website. 

In [51]:
def book_scrape(url):
    titles = []
    ratings = []
    prices = []
    availabilities = []
    urls = [url]
    for i in range(1, 51):
        this_html = requests.get("http://" + url + f"/catalogue/page-{i}.html")
        soup = BeautifulSoup(this_html.content, 'html.parser')
        titles += retrieve_titles(soup)
        ratings += retrieve_ratings(soup)
        prices += retrieve_prices(soup)
        availabilities += retrieve_availabilities(soup)
    return titles, ratings, prices, availabilities

titlez, ratingz, pricez, availabilitiez = book_scrape("books.toscrape.com")

In [58]:
df = pd.DataFrame([titlez, ratingz, pricez, availabilitiez]).transpose()
df.columns = ['Title', 'Star_Rating', 'Price_(pounds)', 'Availability']
df.sort_values(by='Title')

,Title,Star_Rating,Price_(pounds),Availability
491,"""Most Blessed of the Patriarchs"": Thomas Jeffe...",5,44.48,True
887,#GIRLBOSS,1,50.96,True
43,#HigherSelfie: Wake Up Your Life. Free Your So...,5,23.11,True
691,'Salem's Lot,4,49.56,True
127,(Un)Qualified: How God Uses Broken People to D...,5,54,True
...,...,...,...,...
215,Zealot: The Life and Times of Jesus of Nazareth,3,24.7,True
692,Zero History (Blue Ant #3),1,34.77,True
445,"Zero to One: Notes on Startups, or How to Buil...",3,34.06,True
86,orange: The Complete Collection 1 (orange: The...,1,48.41,True


## Level-Up: Write a new version of the script you just wrote. 

If you used URL hacking to generate each successive page URL, instead write a function that retrieves the link from the `"next"` button at the bottom of the page. Conversely, if you already used this approach above, use URL-hacking (arguably the easier of the two methods in this case).

In [76]:
home_url = "http://books.toscrape.com"

#test_html = requests.get("http://books.toscrape.com/catalogue/page-50.html")
#soup1 = BeautifulSoup(test_html.content, 'html.parser')
#print(soup1.find('li', class_='next').find('a').attrs['href'])

def get_soup(url):
    return BeautifulSoup(requests.get(url).content, 'html.parser')

def scrape_all(url, titles=[], ratings=[], prices=[], avails=[]):
    all_urls = [url]
    this_soup = get_soup(all_urls[-1])
    titles += retrieve_titles(this_soup)
    ratings += retrieve_ratings(this_soup)
    prices += retrieve_prices(this_soup)
    avails += retrieve_availabilities(this_soup)
    while this_soup.find('li', class_='next'):
        next_url = this_soup.find('li', class_='next').find('a').attrs['href']
        all_urls.append("http://books.toscrape.com/catalogue/page-" + next_url.split('-')[-1])
        next_soup = get_soup(all_urls[-1])
        titles += retrieve_titles(next_soup)
        ratings += retrieve_ratings(next_soup)
        prices += retrieve_prices(next_soup)
        avails += retrieve_availabilities(next_soup)
        this_soup = next_soup
    return titles, ratings, prices, avails
    

all_titles, all_ratings, all_prices, all_avails = scrape_all(home_url)

df2 = pd.DataFrame([all_titles, all_ratings, all_prices, all_avails]).transpose()
df2.columns = ['Title', 'Star_Rating', 'Price_(pounds)', 'Availability']
df2.head(50)

,Title,Star_Rating,Price_(pounds),Availability
0,A Light in the Attic,3,51.77,True
1,Tipping the Velvet,1,53.74,True
2,Soumission,1,50.1,True
3,Sharp Objects,4,47.82,True
4,Sapiens: A Brief History of Humankind,5,54.23,True
5,The Requiem Red,1,22.65,True
6,The Dirty Little Secrets of Getting Your Dream...,4,33.34,True
7,The Coming Woman: A Novel Based on the Life of...,3,17.93,True
8,The Boys in the Boat: Nine Americans and Their...,4,22.6,True
9,The Black Maria,1,52.15,True


## Summary

Well done! You just completed your first full web scraping project! You're ready to start harnessing the power of the web!